# create a list of master files for analysis

In [4]:
import glob
import pandas as pd
import json
import pathlib
import hashlib
import json
import os
import glob
import datetime
# import shutil
import zipfile
ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")
ts

'20240623_111843_065477'

In [5]:
d_kikin = "/mnt/d/kikin/"

In [6]:
zs = glob.glob(d_kikin + "**/*.zip", recursive=True)
zs[:3]

['/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tensuhyo/ikashika/ikashika_001.files/i_kaiteinaiyo_20240306.zip',
 '/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tensuhyo/ikashika/ikashika_001.files/i_kaiteinaiyo_20240321.zip',
 '/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tensuhyo/ikashika/ikashika_001.files/i_kaiteinaiyo_20240329.zip']

文字化けに対応しつつ解凍

In [7]:
for f in zs:
    # print(f)
    f_out = f.replace(".zip", "")
    # print(f_out)
    with zipfile.ZipFile(f) as z:
        for info in z.infolist():
            info.filename = info.orig_filename.encode('cp437').decode('cp932')
            if os.sep != "/" and os.sep in info.filename:
                info.filename = info.filename.replace(os.sep, "/")
            z.extract(info,f_out)
    
    # break

In [14]:
fs = glob.glob(d_kikin + "**/*.csv", recursive=True)
fs[:3]

['/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tensuhyo/ikashika/ikashika_001.files/osirase_04/サンプル/05算定回数テーブル.csv',
 '/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tensuhyo/ikashika/ikashika_001.files/tensuhyo_02_20120213/tensuhyo_02/01補助マスターテーブル.csv',
 '/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tensuhyo/ikashika/ikashika_001.files/tensuhyo_02_20120213/tensuhyo_02/02包括テーブル.csv']

In [15]:
def get_meta_dict(fs):
    d_meta = dict()
    for f in fs:
        d_meta[f] = dict()
        with open(f,'rb') as file:
            BinaryData = file.read()
            d_meta[f]["sha256"] = hashlib.sha256(BinaryData).hexdigest()
            d_meta[f]["md5"] = hashlib.md5(BinaryData).hexdigest()

        s = os.path.getsize(f)
        d_meta[f]["size"] = str(s/(2**10)) + "kB"

        t = os.path.getmtime(f)
        d_meta[f]["lastfmodification"] = str(datetime.datetime.fromtimestamp(t))

        t = os.path.getctime(f)
        d_meta[f]["filecreation"] = str(datetime.datetime.fromtimestamp(t))

    return d_meta

ファイルのパス、親フォルダなどを得る関数  
正直この処理は美しくないと思う

In [16]:
def get_file_atr_ds(ds, atr):
    res = []
    for f in ds:
        p = pathlib.Path(f)
        if atr == "name":
            res.append(p.name)
        elif atr == "par":
            res.append(p.parent.name)
        elif atr == "gpar":
            res.append(p.parent.parent.name)
        else:
            res.append("")
    return res

## note
以下の系譜のR0504について、日付情報がルール逸脱。  
美しくない個別処理がある。  
増えるようなら処理を考える。  
www.ssk.or.jp/seikyushiharai/tensuhyo/kihonmasta/kihonmasta_02.files/h-3_ALL2023201_R0504.csv


In [17]:
df = pd.DataFrame({"filepath" : fs}) \
    .assign(filename = lambda df:get_file_atr_ds(df.filepath, "name"))\
    .assign(parent = lambda df:get_file_atr_ds(df.filepath, "par"))\
    .assign(gparent = lambda df:get_file_atr_ds(df.filepath, "gpar")) \
    .assign(bearfile = lambda df:df.filename.str.replace(".csv", "",regex=False)) \
    .assign(timeinfo = lambda df:df.bearfile.str.extract('([0-9R_]*$)')) \
    .assign(prefix = lambda df:df.bearfile.str.replace('([0-9_]*$)', "",regex=True)) \
    .assign(ymdinfo = lambda df:df.timeinfo.str.replace('^_', "",regex=True).str[:8]\
               .str.replace('2023201_$', "20230201",regex=True)) \
    .assign(postfix = lambda df:df.timeinfo.str.replace('^_', "",regex=True).str[8:]) \
    .assign(is_kaitei = lambda df:df.bearfile.str.contains("kaitei")) \
    .assign(ymd = lambda df:pd.to_datetime(df.ymdinfo, errors="coerce"))\
    .assign(next_month_day1 = lambda df:df.ymd + pd.offsets.MonthBegin(1)) \
    .assign(next_month_day2 = lambda df:df.next_month_day1 + datetime.timedelta(days =1))
df.head()
# .query("filepath == '/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tensuhyo/kihonmasta/kihonmasta_02.files/h-3_ALL2023201_R0504.csv'")\


,filepath,filename,parent,gparent,bearfile,timeinfo,prefix,ymdinfo,postfix,is_kaitei,ymd,next_month_day1,next_month_day2
0,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,05算定回数テーブル.csv,サンプル,osirase_04,05算定回数テーブル,,05算定回数テーブル,,,False,NaT,NaT,NaT
1,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,01補助マスターテーブル.csv,tensuhyo_02,tensuhyo_02_20120213,01補助マスターテーブル,,01補助マスターテーブル,,,False,NaT,NaT,NaT
2,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,02包括テーブル.csv,tensuhyo_02,tensuhyo_02_20120213,02包括テーブル,,02包括テーブル,,,False,NaT,NaT,NaT
3,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,03-1背反テーブル1.csv,tensuhyo_02,tensuhyo_02_20120213,03-1背反テーブル1,1,03-1背反テーブル,1,,False,NaT,NaT,NaT
4,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,03-2背反テーブル2.csv,tensuhyo_02,tensuhyo_02_20120213,03-2背反テーブル2,2,03-2背反テーブル,2,,False,NaT,NaT,NaT


In [18]:
df.shape

(1541, 13)

In [19]:
%%time
# l_meta = []
d_meta = get_meta_dict(df.filepath)

df_meta = pd.DataFrame().from_dict(d_meta, orient="index")\
    .reset_index(drop=False)\
    .rename(columns={
    "index":"filepath"
})
df_meta.head()
# for r in df.itertuples():
#     print(r.filepath)

#     l_meta.append(d_meta)

CPU times: user 2.75 s, sys: 1.11 s, total: 3.86 s
Wall time: 33.8 s


,filepath,sha256,md5,size,lastfmodification,filecreation
0,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,890542a2abd7d040ab78783bb838af1c456dcdd54e314e...,59b395aa7c467f838d818cd0bf2580cd,10.673828125kB,2024-06-23 11:19:13.768090,2024-06-23 11:19:13.768090
1,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,38cfc25516491e252aef17f93ef4fa4de0ca64ee093485...,cf29e86e3c71970e1f4f88e1e74f5379,886.939453125kB,2024-06-23 11:19:13.811405,2024-06-23 11:19:13.811405
2,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,2b939f604846452eed5c891b4ad97d2b8d9633aa0d9f06...,70d068ccdf6093f9b24b963bb28062e1,6001.912109375kB,2024-06-23 11:19:13.940408,2024-06-23 11:19:13.940408
3,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,e9abb2b236605c230435dafe08e09bbf93941b402582a8...,9443610957dd09433753eb9230b2e1bf,578.05078125kB,2024-06-23 11:19:13.953470,2024-06-23 11:19:13.953470
4,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,92000a77113ec6044414d4f6a325a44463d850db653205...,fcb7bb8a12b2244dc42dffb0ed5ad644,392.81640625kB,2024-06-23 11:19:13.963620,2024-06-23 11:19:13.963620


In [20]:
df_ = pd.merge(df, df_meta, on="filepath")
df_.shape

(1541, 18)

In [21]:
f_x = "./masterfile_{0}.xlsx".format(ts)
f_pk = "./masterfile_{0}.pkl".format(ts)
f_x, f_pk

('./masterfile_20240623_111843_065477.xlsx',
 './masterfile_20240623_111843_065477.pkl')

In [22]:
df_.to_excel(f_x)

In [23]:
df_.to_pickle(f_pk)

In [24]:

# today = datetime.datetime.today()
today = datetime.datetime.fromisoformat("2024-06-23")
# start_day = "2019-06-01"
start_day = "2010-01-01"
max_proc_date = 1e+4
# start_day = "2020-06-01"
print(today.date())
df_dt = pd.DataFrame().from_dict({"dt": pd.date_range(start_day, periods=max_proc_date, freq="D")}) \
    .assign(dt_str = lambda df:df.dt.astype(str)) \
    .assign(yy_str = lambda df:df.dt.dt.year.astype(str)) \
    .assign(mm_str = lambda df:df.dt.dt.month.astype(str)) \
    .assign(dd_str = lambda df:df.dt.dt.day.astype(str)) \
    .assign(is_past = lambda df:df.dt < today ) \
    .query("is_past") \
    .sort_values("dt", ascending=True) \
    .reset_index(drop=True)
# df.query("not is_past").head()

2024-06-23


日付テーブル  
この日付分マスターを作る

In [25]:
df_dt.head()

,dt,dt_str,yy_str,mm_str,dd_str,is_past
0,2010-01-01,2010-01-01,2010,1,1,True
1,2010-01-02,2010-01-02,2010,1,2,True
2,2010-01-03,2010-01-03,2010,1,3,True
3,2010-01-04,2010-01-04,2010,1,4,True
4,2010-01-05,2010-01-05,2010,1,5,True


In [26]:
fdt_x = "./datefile_{0}.xlsx".format(ts)
fdt_pk = "./datefile_{0}.pkl".format(ts)
fdt_x, fdt_pk

('./datefile_20240623_111843_065477.xlsx',
 './datefile_20240623_111843_065477.pkl')

In [27]:
df_dt.to_excel(fdt_x)
df_dt.to_pickle(fdt_pk)